In [1]:
import findspark
findspark = findspark.init()

import os
import datetime
import time
import pandas as pd
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark.sql import Row
import pyspark.sql.types as T


/Users/aravind_jarpala/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/aravind_jarpala/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:

spark = SparkSession.builder. \
        appName('FileFormatComparison'). \
        config('spark.driver.host', 'localhost'). \
        config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.5.1"). \
        getOrCreate()

26/01/10 09:56:02 WARN Utils: Your hostname, Aravinds-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.102 instead (on interface en0)
26/01/10 09:56:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/aravind_jarpala/.ivy2/cache
The jars for the packages stored in: /Users/aravind_jarpala/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-38998e52-71c5-48f1-a213-df0b5158b4f8;1.0
	confs: [default]


:: loading settings :: url = jar:file:/Users/aravind_jarpala/opt/anaconda3/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.spark#spark-avro_2.12;3.5.1 in central
	found org.tukaani#xz;1.9 in central
downloading https://repo1.maven.org/maven2/org/apache/spark/spark-avro_2.12/3.5.1/spark-avro_2.12-3.5.1.jar ...
	[SUCCESSFUL ] org.apache.spark#spark-avro_2.12;3.5.1!spark-avro_2.12.jar (122ms)
downloading https://repo1.maven.org/maven2/org/tukaani/xz/1.9/xz-1.9.jar ...
	[SUCCESSFUL ] org.tukaani#xz;1.9!xz.jar (81ms)
:: resolution report :: resolve 1853ms :: artifacts dl 205ms
	:: modules in use:
	org.apache.spark#spark-avro_2.12;3.5.1 from central in [default]
	org.tukaani#xz;1.9 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   2   |   2   |   0   ||   2   |   2   |
	--------------------------------------

26/01/10 09:56:16 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [3]:
num_rows = 10000000

df = spark.range(0, num_rows)

for i in range(1, 10):
    if i % 2 == 0:
        df = df.withColumn(f'int_col_{i}', (rand() * 100).cast(T.IntegerType()))
    else:
        df = df.withColumn(f'string_col_{i}', (rand() * num_rows).cast(T.IntegerType()).cast('string'))

df.count()

10000000

In [4]:
df.show(10, truncate=False)

+---+------------+---------+------------+---------+------------+---------+------------+---------+------------+
|id |string_col_1|int_col_2|string_col_3|int_col_4|string_col_5|int_col_6|string_col_7|int_col_8|string_col_9|
+---+------------+---------+------------+---------+------------+---------+------------+---------+------------+
|0  |356815      |45       |9743807     |75       |3945915     |85       |6678683     |52       |4395191     |
|1  |7880809     |16       |3471533     |53       |9429465     |64       |6960714     |52       |3936433     |
|2  |9024670     |62       |7883132     |87       |6461195     |49       |2761681     |87       |786446      |
|3  |5812394     |20       |6154180     |52       |8400248     |35       |5602074     |25       |8520916     |
|4  |6784449     |87       |3130471     |33       |5542420     |25       |5486972     |55       |5677606     |
|5  |9998366     |36       |8425499     |87       |2426266     |52       |8955906     |61       |7348519     |
|

In [5]:
base_path = f"data-formats/tmp"

#write as different file formats
df.write.mode("overwrite").csv(f"{base_path}/csv")
df.write.mode("overwrite").parquet(f"{base_path}/parquet")
df.write.mode("overwrite").format("avro").save(f"{base_path}/avro")

26/01/10 09:56:43 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


##### File Format Size

In [ ]:
!du -sh data-formats/tmp/*

461M	data-formats/tmp/avro
649M	data-formats/tmp/csv
420M	data-formats/tmp/parquet


26/01/10 13:03:56 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 681595 ms exceeds timeout 120000 ms
26/01/10 13:03:56 WARN SparkContext: Killing executors is not supported by current scheduler.
26/01/10 13:04:00 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$